In [1]:
# Import libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_percentage_error
from deap import base, creator, tools, algorithms

# Load dataset
df = pd.read_csv("final_processed_data.csv", low_memory=False)

# Prepare data (selected important features based on previous experiments)
FEATURES = ['TOTAL_KM', 'QTY_LOADS', 'QTY_DELIVERIES',
            'COD_DP_MEAN_PRICE_PER_KM', 'COD_LP_MEAN_PRICE_PER_KM',
            'START_DELIVERY_TIME_MEAN_PRICE_PER_KM', 'ENTRY_WEEKDAY_MEAN_PRICE_PER_KM',
            'HU_KM_PERC', 'TEMP_MIN', 'TEMP_MAX']

X = df[FEATURES].values
y = df['EUR'].values

# Set up DEAP evolutionary algorithm
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # minimize MAPE
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

# Define hyperparameter search space
HYPERPARAMETER_BOUNDS = {
    'n_estimators': (50, 1000),
    'learning_rate': (0.01, 0.3),
    'max_depth': (3, 15),
    'subsample': (0.5, 1.0),
    'min_samples_split': (2, 10),
}

# Function to generate random individual
def generate_individual():
    return creator.Individual([
        np.random.randint(*HYPERPARAMETER_BOUNDS['n_estimators']),
        np.random.uniform(*HYPERPARAMETER_BOUNDS['learning_rate']),
        np.random.randint(*HYPERPARAMETER_BOUNDS['max_depth']),
        np.random.uniform(*HYPERPARAMETER_BOUNDS['subsample']),
        np.random.randint(*HYPERPARAMETER_BOUNDS['min_samples_split']),
    ])

toolbox.register("individual", generate_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Evaluation function for an individual (MAPE with TimeSeriesSplit validation)
def eval_gb(individual):
    # Ensure hyperparameters meet valid requirements
    params = {
        'n_estimators': max(50, int(individual[0])),
        'learning_rate': min(max(float(individual[1]), 0.01), 0.3),
        'max_depth': max(1, int(round(individual[2]))),
        'subsample': min(max(float(individual[3]), 0.5), 1.0),
        'min_samples_split': max(2, int(round(individual[4]))),
        'random_state': 42
    }

    model = GradientBoostingRegressor(**params)
    tscv = TimeSeriesSplit(n_splits=5)

    mape_scores = []

    for train_index, test_index in tscv.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        mape = mean_absolute_percentage_error(y_test, y_pred)
        mape_scores.append(mape)

    return (np.mean(mape_scores),)

toolbox.register("evaluate", eval_gb)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", tools.cxUniform, indpb=0.5)

# Custom mutation function with constraints
def custom_mutation(individual, indpb):
    bounds = list(HYPERPARAMETER_BOUNDS.values())
    for i in range(len(individual)):
        if np.random.rand() < indpb:
            if isinstance(bounds[i][0], int):
                individual[i] = int(np.clip(individual[i] + np.random.randint(-50, 50), bounds[i][0], bounds[i][1]))
            else:
                individual[i] = float(np.clip(individual[i] + np.random.uniform(-0.05, 0.05), bounds[i][0], bounds[i][1]))
    return individual,

toolbox.register("mutate", custom_mutation, indpb=0.3)

# Evolutionary algorithm parameters
population_size = 20
num_generations = 10

# Run the evolutionary algorithm
population = toolbox.population(n=population_size)
hall_of_fame = tools.HallOfFame(1)

# Statistics
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("min", np.min)

population, logbook = algorithms.eaSimple(
    population, 
    toolbox, 
    cxpb=0.6, 
    mutpb=0.3, 
    ngen=num_generations, 
    stats=stats, 
    halloffame=hall_of_fame,
    verbose=True
)

# Output best hyperparameters
best_individual = hall_of_fame[0]
best_params = {
    'n_estimators': int(best_individual[0]),
    'learning_rate': best_individual[1],
    'max_depth': int(round(best_individual[2])),
    'subsample': best_individual[3],
    'min_samples_split': int(round(best_individual[4]))
}

print("Best hyperparameters:", best_params)
print("Best MAPE score:", best_individual.fitness.values[0])

# Save logbook for visualization
log_df = pd.DataFrame(logbook)
log_df.to_csv('logbook.csv', index=False)

gen	nevals	avg      	min      
0  	20    	0.0660847	0.0634888
1  	15    	0.064959 	0.0630956
2  	17    	0.0650351	0.0626793
3  	16    	0.0637343	0.0629955
4  	16    	0.0632778	0.0629254
5  	14    	0.0631113	0.0629067
6  	14    	0.0632775	0.0629067
7  	16    	0.0630837	0.0629067
8  	16    	0.0629637	0.0629067
9  	12    	0.0632499	0.0629067
10 	13    	0.0630356	0.0627899
Best hyperparameters: {'n_estimators': 581, 'learning_rate': 0.14215770603493744, 'max_depth': 4, 'subsample': 0.9811039617263888, 'min_samples_split': 10}
Best MAPE score: 0.06267928521999214
